In [ ]:
'Heatmaps'

In [6]:
import os
import glob
import pandas as pd
import re
import plotly.express as px
import numpy as np

def render_heatmap_from_data(heatmap_data, filepath=None, save_dir=None):
    heatmap_matrix = heatmap_data.pivot_table(index='Mutation', columns='Position', values='LCS_Ratio', aggfunc=np.mean)

    title = "LCS Ratio for Each Mutation"
    filename = "heatmap.png"
    if filepath:
        basename = os.path.basename(filepath)
        match = re.match(r"([A-Z0-9]+)_(\d+)_heatmap_data\.csv", basename)
        if match:
            uniprot_id = match.group(1)
            position = match.group(2)
            title = f"Normalized LCS values for {uniprot_id} at position {position}"
            filename = f"{uniprot_id}_{position}_heatmap.png"

    fig = px.imshow(
        heatmap_matrix,
        labels={'x': 'Mutation Position', 'y': 'Mutated Amino Acid', 'color': 'LCS Ratio'},
        color_continuous_scale='Temps',
        title=title
    )

    fig.update_layout(
        xaxis=dict(tickmode='array', tickvals=list(range(1, 16)), ticktext=list(range(-7, 8))),
        yaxis=dict(tickmode='linear', dtick=1),
        coloraxis_colorbar=dict(
            tickvals=[0, 0.25, 0.5, 0.75, 1],
            ticktext=["0.0", "0.25", "0.5", "0.75", "1.0"],
            title="LCS Ratio"
        ),
        plot_bgcolor='white'
    )

    if save_dir:
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, filename)
        fig.write_image(save_path)
        print(f"Saved heatmap to {save_path}")
    else:
        fig.show()

def render_all_heatmaps(folder_path, save_dir=None):
    pattern = os.path.join(folder_path, "*_heatmap_data.csv")
    csv_files = glob.glob(pattern)

    if not csv_files:
        print(f"No heatmap data files found in {folder_path}")
        return

    for file in csv_files:
        try:
            df = pd.read_csv(file)
            print(f"\nRendering heatmap for: {os.path.basename(file)}")
            render_heatmap_from_data(df, filepath=file, save_dir=save_dir)
        except Exception as e:
            print(f"❌ Error rendering {file}: {e}")

# Updated path to read heatmap CSV files from heatmaps folder
heatmap_folder_path = r"C:\Users\ricea\OneDrive\Documents\GitHub\Annabel-BSc-Project\heatmaps"

# Save images in the same folder (or change this to a subfolder if preferred)
heatmap_save_dir = heatmap_folder_path  # or e.g. os.path.join(heatmap_folder_path, "images")

print("🔵 Rendering all heatmaps from heatmaps folder...")
render_all_heatmaps(heatmap_folder_path, save_dir=heatmap_save_dir)


🔵 Rendering all heatmaps from heatmaps folder...

Rendering heatmap for: O00571_323_heatmap_data.csv


C:\Users\ricea\AppData\Local\Temp\ipykernel_4920\1487870648.py:9: FutureWarning:

The provided callable <function mean at 0x00000129DCBE47C0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.



Saved heatmap to C:\Users\ricea\OneDrive\Documents\GitHub\Annabel-BSc-Project\heatmaps\O00571_323_heatmap.png

Rendering heatmap for: O00571_489_heatmap_data.csv


C:\Users\ricea\AppData\Local\Temp\ipykernel_4920\1487870648.py:9: FutureWarning:

The provided callable <function mean at 0x00000129DCBE47C0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.



Saved heatmap to C:\Users\ricea\OneDrive\Documents\GitHub\Annabel-BSc-Project\heatmaps\O00571_489_heatmap.png

Rendering heatmap for: O14578_307_heatmap_data.csv


...

Saved heatmap to C:\Users\ricea\OneDrive\Documents\GitHub\Annabel-BSc-Project\heatmaps\Q9BXW9_222_heatmap.png

Rendering heatmap for: Q9H211_491_heatmap_data.csv


C:\Users\ricea\AppData\Local\Temp\ipykernel_4920\1487870648.py:9: FutureWarning:

The provided callable <function mean at 0x00000129DCBE47C0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.



Saved heatmap to C:\Users\ricea\OneDrive\Documents\GitHub\Annabel-BSc-Project\heatmaps\Q9H211_491_heatmap.png

Rendering heatmap for: Q9NYP3_542_heatmap_data.csv


C:\Users\ricea\AppData\Local\Temp\ipykernel_4920\1487870648.py:9: FutureWarning:

The provided callable <function mean at 0x00000129DCBE47C0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.



Saved heatmap to C:\Users\ricea\OneDrive\Documents\GitHub\Annabel-BSc-Project\heatmaps\Q9NYP3_542_heatmap.png
